# Data Quality notebook

The purpose of this notebook is to perform the Data Quality evaluation in the datasets

In [1]:
import pandas as pd
import numpy as np

# I 94 Aux Database

In [2]:
i94auxdb_file = open('Data/I94_SAS_Labels_Descriptions.SAS')
filecontent = i94auxdb_file.readlines()

In [3]:
country_codes = filecontent[9:298]
country_codes_dic = {}
for rec in country_codes:
    country_codes_key = int(rec.split(' = ')[0].strip())
    country_codes_val = rec.split(' = ')[1].replace("'", "").strip("\n").strip(";").strip()
    country_codes_dic[country_codes_key] = country_codes_val

In [4]:
port_codes = filecontent[302:962]
port_codes_dic = {}
for rec in port_codes:
    port_codes_key = rec.split('\t=\t')[0].strip().replace("'", "")
    port_codes_val = rec.split('\t=\t')[1].replace("'", "").strip()
    port_codes_dic[port_codes_key] = port_codes_val

In [5]:
port_codes_dic

{'ALC': 'ALCAN, AK',
 'ANC': 'ANCHORAGE, AK',
 'BAR': 'BAKER AAF - BAKER ISLAND, AK',
 'DAC': 'DALTONS CACHE, AK',
 'PIZ': 'DEW STATION PT LAY DEW, AK',
 'DTH': 'DUTCH HARBOR, AK',
 'EGL': 'EAGLE, AK',
 'FRB': 'FAIRBANKS, AK',
 'HOM': 'HOMER, AK',
 'HYD': 'HYDER, AK',
 'JUN': 'JUNEAU, AK',
 '5KE': 'KETCHIKAN, AK',
 'KET': 'KETCHIKAN, AK',
 'MOS': 'MOSES POINT INTERMEDIATE, AK',
 'NIK': 'NIKISKI, AK',
 'NOM': 'NOM, AK',
 'PKC': 'POKER CREEK, AK',
 'ORI': 'PORT LIONS SPB, AK',
 'SKA': 'SKAGWAY, AK',
 'SNP': 'ST. PAUL ISLAND, AK',
 'TKI': 'TOKEEN, AK',
 'WRA': 'WRANGELL, AK',
 'HSV': 'MADISON COUNTY - HUNTSVILLE, AL',
 'MOB': 'MOBILE, AL',
 'LIA': 'LITTLE ROCK, AR (BPS)',
 'ROG': 'ROGERS ARPT, AR',
 'DOU': 'DOUGLAS, AZ',
 'LUK': 'LUKEVILLE, AZ',
 'MAP': 'MARIPOSA AZ',
 'NAC': 'NACO, AZ',
 'NOG': 'NOGALES, AZ',
 'PHO': 'PHOENIX, AZ',
 'POR': 'PORTAL, AZ',
 'SLU': 'SAN LUIS, AZ',
 'SAS': 'SASABE, AZ',
 'TUC': 'TUCSON, AZ',
 'YUI': 'YUMA, AZ',
 'AND': 'ANDRADE, CA',
 'BUR': 'BURBANK, CA',
 '

In [6]:
port_codes_df = pd.DataFrame(port_codes_dic.items(), columns=['CODE', 'CITY'])
country_codes_df = pd.DataFrame(country_codes_dic.items(), columns=['CODE', 'CITY'])

In [7]:
port_codes_df

,CODE,CITY
0,ALC,"ALCAN, AK"
1,ANC,"ANCHORAGE, AK"
2,BAR,"BAKER AAF - BAKER ISLAND, AK"
3,DAC,"DALTONS CACHE, AK"
4,PIZ,"DEW STATION PT LAY DEW, AK"
...,...,...
655,ADU,No PORT Code (ADU)
656,AKT,No PORT Code (AKT)
657,LIT,No PORT Code (LIT)
658,A2A,No PORT Code (A2A)


In [8]:
port_codes_df.to_csv('port_codes.csv')
country_codes_df.to_csv('country_codes.csv')

## Create Arrivals Table

In [14]:
i94_fname = 'Data/18-83510-I94-Data-2016/i94_jan16_sub.sas7bdat'
i94_df = pd.read_sas(i94_fname, 'sas7bdat', encoding="ISO-8859-1")

In [15]:
df_sasdatadict = pd.read_excel('Data/DataDict_SASfile.xlsx', sheet_name=0) 
newSAScolumns = df_sasdatadict.loc[0, :].values.tolist()
i94_df.columns = newSAScolumns

In [16]:
Arrivals_df = i94_df[['ID','PORTCODE','AIRLINECODE','FLIGHTNUMBER','DEPARTUREDATE','ARRIVALDATE','DEPARTUREFLAG','ARRIVALFLAG']].copy()
Arrivals_df['MUNICIPALITY'] = Arrivals_df['PORTCODE'].map(port_codes_dic).to_list()

In [17]:
Arrivals_df['CITYNAME'] =  Arrivals_df['MUNICIPALITY'].str.split(',',expand=True)[0]

In [19]:
i94_df.sort_values('ID')

,ID,YEAR,MONTH,COUNTRYCODEPROCESSING_CIT,COUNTRYCODEPROCESSING_RES,PORTCODE,ARRIVALDATE,MODAL,STATE,DEPARTUREDATE,...,UPDATEFLAG,MATCHARRDEP,YEARBIRTH,ADDMITTEDUNTIL,GENDER,INS_NUM,AIRLINECODE,ADMINNUMBER,FLIGHTNUMBER,VISATYPE
0,7.0,2016.0,1.0,101.0,101.0,BOS,20465.0,1.0,MA,NaN,...,NaN,NaN,1996.0,D/S,M,NaN,LH,3.466083e+08,424,F1
1,8.0,2016.0,1.0,101.0,101.0,BOS,20465.0,1.0,MA,NaN,...,NaN,NaN,1996.0,D/S,M,NaN,LH,3.466276e+08,424,F1
2,9.0,2016.0,1.0,101.0,101.0,BOS,20469.0,1.0,CT,20480.0,...,NaN,M,1999.0,07152016,F,NaN,AF,3.810924e+08,338,B2
3,10.0,2016.0,1.0,101.0,101.0,BOS,20469.0,1.0,CT,20499.0,...,NaN,M,1971.0,07152016,F,NaN,AF,3.810879e+08,338,B2
4,11.0,2016.0,1.0,101.0,101.0,BOS,20469.0,1.0,CT,20499.0,...,NaN,M,2004.0,07152016,M,NaN,AF,3.810787e+08,338,B2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2839548,6148372.0,2016.0,1.0,438.0,438.0,MET,20480.0,3.0,CA,20521.0,...,NaN,M,1961.0,04252016,F,NaN,NaN,1.758801e+10,LAND,WT
2797111,6148373.0,2016.0,1.0,438.0,438.0,XXX,20478.0,2.0,FL,NaN,...,NaN,M,1986.0,07222016,M,3148,NaN,1.356867e+10,NaN,B1
2797112,6148374.0,2016.0,1.0,464.0,464.0,XXX,20478.0,2.0,FL,NaN,...,NaN,M,1989.0,07222016,F,3148,NaN,1.356867e+10,NaN,B1
2797113,6148393.0,2016.0,1.0,107.0,107.0,ERI,20462.0,1.0,PA,NaN,...,NaN,NaN,1994.0,D/S,F,NaN,B3D,7.997488e+08,NONE,F1


## Create Immigration_Details table

In [20]:
Immigrant_Details_df = i94_df[['ID','COUNTRYCODEPROCESSING_CIT','COUNTRYCODEPROCESSING_RES','VISACODE','WHEREVISAISSUED','ADDMITTEDUNTIL','VISATYPE']].copy()
Immigrant_Details_df['CITIZENSHIP'] = Immigrant_Details_df['COUNTRYCODEPROCESSING_CIT'].map(country_codes_dic).to_list()
Immigrant_Details_df['RESIDENCY'] = Immigrant_Details_df['COUNTRYCODEPROCESSING_RES'].map(country_codes_dic).to_list()

In [21]:
Immigrant_Details_df.sort_values('ID')

,ID,COUNTRYCODEPROCESSING_CIT,COUNTRYCODEPROCESSING_RES,VISACODE,WHEREVISAISSUED,ADDMITTEDUNTIL,VISATYPE,CITIZENSHIP,RESIDENCY
0,7.0,101.0,101.0,3.0,NaN,D/S,F1,ALBANIA,ALBANIA
1,8.0,101.0,101.0,3.0,NaN,D/S,F1,ALBANIA,ALBANIA
2,9.0,101.0,101.0,2.0,NaN,07152016,B2,ALBANIA,ALBANIA
3,10.0,101.0,101.0,2.0,NaN,07152016,B2,ALBANIA,ALBANIA
4,11.0,101.0,101.0,2.0,NaN,07152016,B2,ALBANIA,ALBANIA
...,...,...,...,...,...,...,...,...,...
2839548,6148372.0,438.0,438.0,2.0,NaN,04252016,WT,AUSTRALIA,AUSTRALIA
2797111,6148373.0,438.0,438.0,1.0,PAR,07222016,B1,AUSTRALIA,AUSTRALIA
2797112,6148374.0,464.0,464.0,1.0,PAR,07222016,B1,NEW ZEALAND,NEW ZEALAND
2797113,6148393.0,107.0,107.0,3.0,WRW,D/S,F1,POLAND,POLAND


## Create Personal_Demographics table

In [41]:
Personal_Demographics_df = i94_df[['ID','YEARBIRTH','GENDER','OCCUPATIONWILLBEPERFORMEDINUS']].copy()

In [45]:
Personal_Demographics_df

,ID,YEARBIRTH,GENDER,OCCUPATIONWILLBEPERFORMEDINUS
0,6.0,1979.0,NaN,NaN
1,7.0,1991.0,M,NaN
2,15.0,1961.0,M,NaN
3,16.0,1988.0,NaN,NaN
4,17.0,2012.0,NaN,NaN
...,...,...,...,...
3096308,625229.0,1980.0,NaN,NaN
3096309,1972204.0,1980.0,F,NaN
3096310,4249448.0,1993.0,F,NaN
3096311,5658953.0,1959.0,M,NaN


## Create Cities_Temp table

In [270]:
Cities_Temp_df = pd.read_csv('Data/GlobalLandTemperaturesByCity.csv')
Cities_Temp_df.columns

Index(['dt', 'AverageTemperature', 'AverageTemperatureUncertainty', 'City',
       'Country', 'Latitude', 'Longitude'],
      dtype='object')

In [271]:
Cities_Temp_df = Cities_Temp_df[Cities_Temp_df['Country']=='United States']
Cities_Temp_df = Cities_Temp_df[['AverageTemperature','City']]
Cities_Temp_min_df = Cities_Temp_df.groupby(['City']).min()
Cities_Temp_max_df = Cities_Temp_df.groupby(['City']).max()
Cities_Temp_min_df = Cities_Temp_min_df.rename(columns={"AverageTemperature": "MinTemperature"})
Cities_Temp_max_df = Cities_Temp_max_df.rename(columns={"AverageTemperature": "MaxTemperature"})

In [272]:
Cities_Temp_df = Cities_Temp_df.groupby(['City']).mean()

In [273]:
Cities_Temp_df = Cities_Temp_df.join(Cities_Temp_min_df)
Cities_Temp_df = Cities_Temp_df.join(Cities_Temp_max_df)

In [274]:
Cities_Temp_df['city'] = Cities_Temp_df.index.str.upper()

In [275]:
Cities_Temp_df

,AverageTemperature,MinTemperature,MaxTemperature,city
City,,,,
Abilene,16.892500,-0.568,32.110,ABILENE
Akron,9.605076,-11.416,28.597,AKRON
Albuquerque,11.135264,-5.073,25.688,ALBUQUERQUE
Alexandria,11.918475,-6.386,30.833,ALEXANDRIA
Allentown,9.523296,-9.162,29.141,ALLENTOWN
...,...,...,...,...
Wichita Falls,16.485163,-3.574,33.778,WICHITA FALLS
Windsor,8.520341,-11.638,28.313,WINDSOR
Winston Salem,14.418861,-2.690,31.475,WINSTON SALEM


In [290]:
#Cities_Temp_df[Cities_Temp_df['city'] == 'NEWARK']
#Cities_Temp_df[Cities_Temp_df['city'].str.startswith('H')]

## Create Cities_Demographics table

In [24]:
citiesdemo_df = pd.read_json('Data/us-cities-demographics.json')

In [29]:
citiesdemo_df['fields'][0:3]

0    {'count': 76402, 'city': 'Newark', 'number_of_...
1    {'count': 1343, 'city': 'Peoria', 'number_of_v...
2    {'count': 2583, 'city': 'O'Fallon', 'number_of...
Name: fields, dtype: object

In [25]:
pd.DataFrame(citiesdemo_df.fields.values.tolist())

,count,city,number_of_veterans,male_population,foreign_born,average_household_size,median_age,state,race,total_population,state_code,female_population
0,76402,Newark,5829.0,138040.0,86253.0,2.73,34.6,New Jersey,White,281913,NJ,143873.0
1,1343,Peoria,6634.0,56229.0,7517.0,2.40,33.1,Illinois,American Indian and Alaska Native,118661,IL,62432.0
2,2583,O'Fallon,5783.0,41762.0,3269.0,2.77,36.0,Missouri,Hispanic or Latino,85032,MO,43270.0
3,70303,Hampton,19638.0,66214.0,6204.0,2.48,35.5,Virginia,Black or African-American,136454,VA,70240.0
4,33630,Lakewood,9988.0,76013.0,14169.0,2.29,37.7,Colorado,Hispanic or Latino,152589,CO,76576.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2886,14735,Boca Raton,4367.0,44760.0,21117.0,2.22,47.3,Florida,Hispanic or Latino,93226,FL,48466.0
2887,28148,Fort Lauderdale,8897.0,93948.0,47582.0,2.38,42.8,Florida,Hispanic or Latino,178587,FL,84639.0
2888,165423,Oxnard,6367.0,101906.0,78678.0,4.08,31.0,California,White,207252,CA,105346.0
2889,62927,Somerville,2103.0,41028.0,22292.0,2.43,31.0,Massachusetts,White,80334,MA,39306.0


In [227]:
citiesdemo_fields_df = pd.DataFrame(citiesdemo_df.fields.values.tolist())

In [228]:
citiesdemo_df = citiesdemo_df.join(citiesdemo_fields_df)

In [229]:
citiesdemo_df['city'] = citiesdemo_df['city'].str.upper()

In [230]:
citiesdemo_pivot = citiesdemo_df[['city','count','race']].pivot_table(index="city", columns="race", values="count", aggfunc=np.sum, fill_value=0)

In [231]:
citiesdemo_df = citiesdemo_df[['city','female_population','male_population','foreign_born','number_of_veterans','total_population','average_household_size']].drop_duplicates()

In [232]:
Cities_Demographics_df = citiesdemo_pivot.merge(citiesdemo_df,left_index=True, right_on='city')

In [233]:
Cities_Demographics_df

,American Indian and Alaska Native,Asian,Black or African-American,Hispanic or Latino,White,city,female_population,male_population,foreign_born,number_of_veterans,total_population,average_household_size
1058,1813,2929,14449,33222,95487,ABILENE,60664.0,65212.0,8129.0,9367.0,125876,2.64
438,1845,9033,66551,3684,129192,AKRON,100667.0,96886.0,10024.0,12878.0,197553,2.24
907,0,10336,6577,34897,63666,ALAFAYA,45760.0,39504.0,15842.0,4176.0,85264,2.94
872,1329,27984,7364,8265,44232,ALAMEDA,40867.0,37747.0,18841.0,4504.0,78614,2.52
44,2056,8740,84743,11151,75528,ALBANY,39414.0,31695.0,861.0,5409.0,71109,2.38
...,...,...,...,...,...,...,...,...,...,...,...,...
752,1112,13981,38731,73608,129492,YONKERS,104538.0,96580.0,61247.0,4801.0,201118,2.80
337,211,17616,1326,10599,49980,YORBA LINDA,36006.0,31960.0,15532.0,3171.0,67966,3.00
342,875,247,31725,7564,33031,YOUNGS,33851.0,30758.0,1058.0,3620.0,64609,2.22
627,2163,15065,1912,20890,48746,YUBA CITY,33290.0,33654.0,18032.0,4706.0,66944,2.90


In [22]:
#Cities_Demographics_df[Cities_Demographics_df['city']=='SAINT PAUL']

In [23]:
#Cities_Demographics_df[Cities_Demographics_df['city']=='SARASOTA']

## Match Arrival City Demographics

In [246]:
Cities_Demographics_df.loc[Cities_Demographics_df['city'] == 'NEWARK', ['city']] = 'NEWARK/TETERBORO'
Cities_Demographics_df.loc[Cities_Demographics_df['city'] == 'WASHINGTON', ['city']] = 'WASHINGTON DC'
Cities_Demographics_df.loc[Cities_Demographics_df['city'] == 'ST PAUL', ['city']] = 'SAINT PAUL'

## Match Arrival City Temperatures

In [285]:
Cities_Temp_df.loc[Cities_Temp_df['city'] == 'NEWARK', ['city']] = 'NEWARK/TETERBORO'
Cities_Temp_df.loc[Cities_Temp_df['city'] == 'WASHINGTON', ['city']] = 'WASHINGTON DC'
Cities_Temp_df.loc[Cities_Temp_df['city'] == 'ST PAUL', ['city']] = 'SAINT PAUL'

In [286]:
Cities_Temp_city_df = Cities_Temp_df['city'].unique()

In [287]:
dif_temp = list(set(Arrivals_Cities).difference(Cities_Temp_city_df))
match_temp = list(set(Arrivals_Cities).intersection(Cities_Temp_city_df))

In [288]:
Arrivals_df[Arrivals_df['CITY'].isin(dif_temp)][['CITY']].value_counts().head(20)

CITY           
HONOLULU           142720
AGANA               80919
DUBLIN              24371
SAIPAN              23628
TORONTO             20886
Abu Dhabi           18151
PALM SPRINGS        18117
FORT MYERS          17514
ST PAUL             16973
KAHULUI - MAUI      13259
NASSAU              13032
BLAINE              11087
SANFORD             10159
SAN JUAN             9144
WEST PALM BEACH      9093
NIAGARA FALLS        6102
MONTREAL             6006
HAMILTON             5207
NORTH CAICOS         5197
CHAMPLAIN            5197
dtype: int64

In [254]:
Arrivals_Cities = Arrivals_df['CITY'].unique()
City_Demo_Cities = Cities_Demographics_df['city'].unique()

In [248]:
dif_demo = list(set(Arrivals_Cities).difference(City_Demo_Cities))
match_demo = list(set(Arrivals_Cities).intersection(City_Demo_Cities))

In [249]:
Arrivals_df[Arrivals_df['CITY'].isin(dif_demo)][['CITY']].value_counts().head(20)

CITY                
HONOLULU                142720
AGANA                    80919
DUBLIN                   24371
SAIPAN                   23628
TORONTO                  20886
Abu Dhabi                18151
PALM SPRINGS             18117
ST PAUL                  16973
KAHULUI - MAUI           13259
NASSAU                   13032
BLAINE                   11087
SANFORD                  10159
NIAGARA FALLS             6102
MONTREAL                  6006
HAMILTON                  5207
CHAMPLAIN                 5197
NORTH CAICOS              5197
KEAHOLE-KONA              4042
PEACE BRIDGE              4035
BRADENTON - SARASOTA      3922
dtype: int64

In [198]:
#Cities_Demographics_df[Cities_Demographics_df['city'].str.startswith('H')]